In [1]:
from casadi import SX, inv, vertcat, horzcat, Function, integrator, sin, cos, diag, skew, substitute, DM , tan

In [2]:
# Equations of motion expressed in BODY

# 6 DOF states vectors
u = SX.sym('u')
v = SX.sym('v')
w = SX.sym('w')
v_nb = vertcat(u, v, w)  # body-fixed linear velocity

p = SX.sym('p')
q = SX.sym('q')
r = SX.sym('r')
w_nb = vertcat(p, q, r)  # body-fixed angular velocity

x_nb = vertcat(v_nb, w_nb)


W = SX.sym('W')  # weight
B = SX.sym('B')  # buoyancy

thet = SX.sym('thet')
phi = SX.sym('phi')
psi = SX.sym('psi')

m = SX.sym('m')  # Mass

I_x = SX.sym('I_x')  # moment of inertia x entry
I_y = SX.sym('I_y')  # moment of inertia y entry
I_z = SX.sym('I_z')  # moment of inertia z entry
I_zx = SX.sym('I_zx')  # product of inertia zx entry
I_yx = SX.sym('I_yx')  # product of inertia yx entry
I_zy = SX.sym('I_zy')  # product of inertia zy entry

x_g = SX.sym('x_g')  # Center of gravity, x-axis
y_g = SX.sym('y_g')  # Center of gravity, y-axis
z_g = SX.sym('z_g')  # Center of gravity, z-axis

x_b = SX.sym('x_b')  # Center of buoyancy, x-axis
y_b = SX.sym('y_b')  # Center of buoyancy, y-axis
z_b = SX.sym('z_b')  # Center of buoyancy, z-axis


X_du = SX.sym('X_du') # Added mass in surge
X_dv = SX.sym('X_dv') # coupled Added mass in surge & sway
X_dw = SX.sym('X_dw') # coupled Added mass in surge & heave
X_dp = SX.sym('X_dp') # coupled Added mass in surge & roll
X_dq = SX.sym('X_dq') # coupled Added mass in surge & pitch
X_dr = SX.sym('X_dr') # coupled Added mass in surge & pitch

Y_du = SX.sym('Y_du') # Added mass in sway & surge
Y_dv = SX.sym('Y_dv') # Added mass in sway
Y_dw = SX.sym('Y_dw') # coupled Added mass in sway & heave
Y_dp = SX.sym('Y_dp') # coupled Added mass in sway & roll
Y_dq = SX.sym('Y_dq') # coupled Added mass in sway & pitch
Y_dr = SX.sym('Y_dr') # coupled Added mass in sway & yaw

Z_du = SX.sym('Z_du') # coupled Added mass in heave & surge
Z_dv = SX.sym('Z_dv') # coupled Added mass in heave & sway
Z_dw = SX.sym('Z_dw') # Added mass in heave
Z_dp = SX.sym('Z_dp') # coupled Added mass in heave & roll
Z_dq = SX.sym('Z_dq') # coupled Added mass in heave & pitch
Z_dr = SX.sym('Z_dr') # coupled Added mass in heave & yaw

K_du = SX.sym('K_du') # coupled Added mass in roll & surge
K_dv = SX.sym('K_dv') # coupled Added mass in roll & sway
K_dw = SX.sym('K_dw') # coupled Added mass in roll & heave
K_dp = SX.sym('K_dp') # Added mass in roll
K_dq = SX.sym('K_dq') # coupled Added mass in roll & pitch
K_dr = SX.sym('K_dr') # coupled Added mass in roll & yaw

M_du = SX.sym('M_du') # coupled Added mass in pitch & surge
M_dv = SX.sym('M_dv') # coupled Added mass in pitch & sway
M_dw = SX.sym('M_dw') # coupled Added mass in pitch & heave
M_dp = SX.sym('M_dp') # coupled Added mass in pitch & roll
M_dq = SX.sym('M_dq') # Added mass in pitch
M_dr = SX.sym('M_dr') # coupled Added mass in pitch & yaw

N_du = SX.sym('N_du') # coupled Added mass in yaw & surge
N_dv = SX.sym('N_dv') # coupled Added mass in yaw & sway
N_dw = SX.sym('N_dw') # coupled Added mass in yaw & heave
N_dp = SX.sym('N_dp') # coupled Added mass in yaw & roll
N_dq = SX.sym('N_dq') # coupled Added mass in yaw & pitch
N_dr = SX.sym('N_dr') # Added mass in yaw

In [3]:
R = SX(3, 3)
R[0,0] = cos(psi)*cos(thet)
R[0,1] = -sin(psi)*cos(phi) + cos(psi)*sin(thet)*sin(phi)
R[0,2] = sin(psi)*cos(phi) + cos(psi)*cos(phi)*sin(thet)
R[1,0] = sin(psi)*cos(thet)
R[1,1] = cos(psi)*cos(phi) + sin(phi)*sin(thet)*sin(psi)
R[1,2] = -cos(psi)*cos(phi) + sin(thet)*sin(psi)*sin(phi)
R[2,0] = -sin(thet)
R[2,1] = cos(thet)*sin(phi)
R[2,2] = cos(thet)*cos(phi)
R


SX(
[[(cos(psi)*cos(thet)), (((cos(psi)*sin(thet))*sin(phi))-(sin(psi)*cos(phi))), ((sin(psi)*cos(phi))+((cos(psi)*cos(phi))*sin(thet)))], 
 [(sin(psi)*cos(thet)), ((cos(psi)*cos(phi))+((sin(phi)*sin(thet))*sin(psi))), (((sin(thet)*sin(psi))*sin(phi))-(cos(psi)*cos(phi)))], 
 [(-sin(thet)), (cos(thet)*sin(phi)), (cos(thet)*cos(phi))]])

In [4]:
T = SX.eye(3)
T[0,1] = sin(phi)*tan(thet)
T[0,2] = cos(phi)*tan(thet)
T[1,1] = cos(phi)
T[1,2] = -sin(phi)
T[2,1] = sin(phi)/cos(thet)
T[2,2] = cos(phi)/cos(thet)


T_1 = SX.eye(3)
T_1[0,2] = -sin(thet)
T_1[1,1] = cos(phi)
T_1[1,2] = cos(thet)*sin(phi)
T_1[2,1] = -sin(phi)
T_1[2,2] = cos(thet)*cos(phi)
T_1

SX(
[[1, 00, (-sin(thet))], 
 [00, cos(phi), (cos(thet)*sin(phi))], 
 [00, (-sin(phi)), (cos(thet)*cos(phi))]])

In [5]:
J = SX.zeros(6, 6)
J[:3,:3] = R
J[3:,3:] = T
J

J_1 = SX.zeros(6, 6)
J_1[:3,:3] = R.T
J_1[3:,3:] = inv(T)
J_1


SX(@1=0, @2=cos(phi), @3=(cos(phi)/cos(thet)), @4=sin(phi), @5=(sin(phi)/cos(thet)), @6=((@2*@3)+(@4*@5)), @7=(cos(phi)*tan(thet)), @8=(sin(phi)*tan(thet)), 
[[(cos(psi)*cos(thet)), (sin(psi)*cos(thet)), (-sin(thet)), @1, @1, @1], 
 [(((cos(psi)*sin(thet))*sin(phi))-(sin(psi)*cos(phi))), ((cos(psi)*cos(phi))+((sin(phi)*sin(thet))*sin(psi))), (cos(thet)*sin(phi)), @1, @1, @1], 
 [((sin(psi)*cos(phi))+((cos(psi)*cos(phi))*sin(thet))), (((sin(thet)*sin(psi))*sin(phi))-(cos(psi)*cos(phi))), (cos(thet)*cos(phi)), @1, @1, @1], 
 [@1, @1, @1, (((@2*@3)+(@4*@5))/@6), (((@5*@7)-(@3*@8))/@6), (-(((@2*@7)+(@4*@8))/@6))], 
 [@1, @1, @1, 00, (@3/@6), (@4/@6)], 
 [@1, @1, @1, 00, (-(@5/@6)), (@2/@6)]])

In [6]:
_M = SX(6, 6)


_M[0, :] = horzcat(m - X_du, -X_dv, -X_dw, -X_dp, m*z_g - X_dq, -m*y_g - X_dr)
_M[1, :] = horzcat(-X_dv, m-Y_dv, -Y_dw, -m*z_g-Y_dp, -Y_dq, m*x_g - Y_dr)
_M[2, :] = horzcat(-X_dw, -Y_dw, m - Z_dw, m*y_g - Z_dp, -m*x_g - Z_dq, -Z_dr)
_M[3, :] = horzcat(-X_dp, -m*z_g-Y_dp, m*y_g - Z_dp, I_x - K_dp, -I_yx - K_dq, -I_zx - K_dr)
_M[4, :] = horzcat(m*z_g - X_dq, -Y_dq, -m*x_g - Z_dq, -I_yx - K_dq, I_y - M_dq, -I_zy - M_dr)
_M[5, :] = horzcat(-m*y_g - X_dr, m*x_g - Y_dr, -Z_dr, -I_zx - K_dr, -I_zy - M_dr, I_z - N_dr)

_M

SX(
[[(m-X_du), (-X_dv), (-X_dw), (-X_dp), ((m*z_g)-X_dq), (-((m*y_g)+X_dr))], 
 [(-X_dv), (m-Y_dv), (-Y_dw), (-((m*z_g)+Y_dp)), (-Y_dq), ((m*x_g)-Y_dr)], 
 [(-X_dw), (-Y_dw), (m-Z_dw), ((m*y_g)-Z_dp), (-((m*x_g)+Z_dq)), (-Z_dr)], 
 [(-X_dp), (-((m*z_g)+Y_dp)), ((m*y_g)-Z_dp), (I_x-K_dp), (-(I_yx+K_dq)), (-(I_zx+K_dr))], 
 [((m*z_g)-X_dq), (-Y_dq), (-((m*x_g)+Z_dq)), (-(I_yx+K_dq)), (I_y-M_dq), (-(I_zy+M_dr))], 
 [(-((m*y_g)+X_dr)), ((m*x_g)-Y_dr), (-Z_dr), (-(I_zx+K_dr)), (-(I_zy+M_dr)), (I_z-N_dr)]])

In [7]:
# assuming Starboard–port symmetrical underwater vehicles yg=0 and Ixy = Iyz = 0
mask = DM([[1, 0, 1, 0, 1, 0],
           [0, 1, 0, 1, 0, 1],
           [1, 0, 1, 0, 1, 0],
           [0, 1, 0, 1, 0, 1],
           [1, 0, 1, 0, 1, 0],
           [0, 1, 0, 1, 0, 1]
           ])

M = _M*mask
M

SX(@1=0, 
[[(m-X_du), @1, (-X_dw), @1, ((m*z_g)-X_dq), @1], 
 [@1, (m-Y_dv), @1, (-((m*z_g)+Y_dp)), @1, ((m*x_g)-Y_dr)], 
 [(-X_dw), @1, (m-Z_dw), @1, (-((m*x_g)+Z_dq)), @1], 
 [@1, (-((m*z_g)+Y_dp)), @1, (I_x-K_dp), @1, (-(I_zx+K_dr))], 
 [((m*z_g)-X_dq), @1, (-((m*x_g)+Z_dq)), @1, (I_y-M_dq), @1], 
 [@1, ((m*x_g)-Y_dr), @1, (-(I_zx+K_dr)), @1, (I_z-N_dr)]])

In [8]:
M11 = M[:3,:3]
M12 = M[:3,3:]
M21 = M[3:,:3]
M22 = M[3:,3:]

C = SX.zeros(6, 6)

C[3:,:3] = -skew(M11@v_nb + M12@w_nb)
C[:3,3:] = -skew(M11@v_nb + M12@w_nb)
C[3:,3:] = -skew(M21@v_nb + M22@w_nb)

C 


SX(@1=0, @2=(m-Z_dw), @3=((m*x_g)+Z_dq), @4=(((@2*w)-(X_dw*u))-(@3*q)), @5=(m-Y_dv), @6=((m*x_g)-Y_dr), @7=((m*z_g)+Y_dp), @8=((@5*v)+((@6*r)-(@7*p))), @9=(m-X_du), @10=((m*z_g)-X_dq), @11=(((@9*u)-(X_dw*w))+(@10*q)), @12=(((@2*w)-(X_dw*u))-(@3*q)), @13=((@5*v)+((@6*r)-(@7*p))), @14=((((m*x_g)-Y_dr)*v)+(((I_z-N_dr)*r)-((I_zx+K_dr)*p))), @15=(((((m*z_g)-X_dq)*u)-(((m*x_g)+Z_dq)*w))+((I_y-M_dq)*q)), @16=(((@9*u)-(X_dw*w))+(@10*q)), @17=((((I_x-K_dp)*p)-((I_zx+K_dr)*r))-(((m*z_g)+Y_dp)*v)), 
[[@1, @1, @1, @1, @12, (-@13)], 
 [@1, @1, @1, (-@12), @1, @16], 
 [@1, @1, @1, @13, (-@16), @1], 
 [@1, @4, (-@8), @1, @14, (-@15)], 
 [(-@4), @1, @11, (-@14), @1, @17], 
 [@8, (-@11), @1, @15, (-@17), @1]])

In [9]:
# Hydrostatics of Submerged Vehicles
# restoring forces

g = SX(6, 1)

g[0, 0] = (W - B)*sin(thet)
g[1, 0] = -(W - B)*cos(thet)*sin(phi)
g[2, 0] = -(W - B)*cos(thet)*cos(phi)
g[3, 0] = -(y_g*W - y_b*B)*cos(thet)*cos(phi) + \
    (z_g*W - z_b*B)*cos(thet)*sin(phi)
g[4, 0] = (z_g*W - z_b*B)*sin(thet) + (x_g*W - x_b*B)*cos(thet)*cos(phi)
g[5, 0] = -(x_g*W - x_b*B)*cos(thet)*sin(phi) - (y_g*W - y_b*B)*sin(thet)
# For neutrally buoyant vehicles W = B

g

SX([((W-B)*sin(thet)), (-(((W-B)*cos(thet))*sin(phi))), (-(((W-B)*cos(thet))*cos(phi))), (((((z_g*W)-(z_b*B))*cos(thet))*sin(phi))-((((y_g*W)-(y_b*B))*cos(thet))*cos(phi))), ((((z_g*W)-(z_b*B))*sin(thet))+((((x_g*W)-(x_b*B))*cos(thet))*cos(phi))), (-(((((x_g*W)-(x_b*B))*cos(thet))*sin(phi))+(((y_g*W)-(y_b*B))*sin(thet))))])

In [10]:
d_R = R@skew(v_nb)
d_T = R@skew(w_nb)
d_J = SX.zeros(6, 6)
d_J[:3,:3] = d_R
d_J[3:,3:] = d_T
d_J

SX(@1=(((cos(psi)*sin(thet))*sin(phi))-(sin(psi)*cos(phi))), @2=((sin(psi)*cos(phi))+((cos(psi)*cos(phi))*sin(thet))), @3=((cos(psi)*cos(phi))+((sin(phi)*sin(thet))*sin(psi))), @4=(((sin(thet)*sin(psi))*sin(phi))-(cos(psi)*cos(phi))), @5=(cos(thet)*sin(phi)), @6=(cos(thet)*cos(phi)), @7=0, @8=(cos(psi)*cos(thet)), @9=(sin(psi)*cos(thet)), @10=sin(thet), 
[[((@1*w)-(@2*v)), ((@2*u)-(@8*w)), ((@8*v)-(@1*u)), @7, @7, @7], 
 [((@3*w)-(@4*v)), ((@4*u)-(@9*w)), ((@9*v)-(@3*u)), @7, @7, @7], 
 [((@5*w)-(@6*v)), ((@10*w)+(@6*u)), (-((@10*v)+(@5*u))), @7, @7, @7], 
 [@7, @7, @7, ((@1*r)-(@2*q)), ((@2*p)-(@8*r)), ((@8*q)-(@1*p))], 
 [@7, @7, @7, ((@3*r)-(@4*q)), ((@4*p)-(@9*r)), ((@9*q)-(@3*p))], 
 [@7, @7, @7, ((@5*r)-(@6*q)), ((@10*r)+(@6*p)), (-((@10*q)+(@5*p)))]])

In [11]:
M_ned = J_1.T@M@J_1

# C_ned = J_1.T(C-M@J_1@d_J)@J_1
g_ned = J_1.T@g
# tau_ned = J_1.T@tau

TypeError: 'SX' object is not callable